### Getting Started With ML Project With MLFLOW

- Installing MLflow.

- Starting a local MLflow Tracking Server.

- Logging and registering a model with MLflow.

- Loading a logged model for inference using MLflow’s pyfunc flavor.

- Viewing the experiment results in the MLflow UI.

In [1]:
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature

In [2]:
import mlflow
from mlflow.models import infer_signature

# print(mlflow.__version__)


In [3]:
## load the dataset
X,y=datasets.load_iris(return_X_y=True)
# split the data into training and test sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20)

# Define the model hyperparameters
params = {"penalty":"l2","solver": "lbfgs", "max_iter": 1000, "multi_class": "auto", "random_state": 8888}

##train the model

lr=LogisticRegression(**params)
lr.fit(X_train,y_train)


c:\Users\nosha\OneDrive\Desktop\New folder\menv\lib\site-packages\sklearn\linear_model\_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,8888
,solver,'lbfgs'
,max_iter,1000
,multi_class,'auto'


In [4]:
## Prediction on the test set
y_pred=lr.predict(X_test)
y_pred

array([2, 1, 0, 1, 1, 1, 1, 1, 2, 1, 0, 2, 0, 2, 0, 0, 1, 2, 2, 0, 0, 2,
       2, 0, 2, 1, 0, 1, 2, 1])

In [5]:
accuracy=accuracy_score(y_test,y_pred)
print(accuracy)

1.0


### MLFLOW Tracking

In [13]:
### MLFLOW tracking
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

## create a new MLFLOW experiment
mlflow.set_experiment("ML20")

## Start an MLFLOW run
with mlflow.start_run():
    ## log the hyperparameters
    mlflow.log_params(params)

    ## Log the accuracy metrics
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    ## Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    ## log the model
    # The `mlflow.sklearn.log_model` call must be inside the `with` block
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart3",
    )


2025/08/26 16:36:41 INFO mlflow.tracking.fluent: Experiment with name 'ML20' does not exist. Creating a new experiment.


2025/08/26 16:36:41 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Successfully registered model 'tracking-quickstart3'.
2025/08/26 16:36:48 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart3, version 1
Created version '1' of model 'tracking-quickstart3'.


🏃 View run overjoyed-grouse-97 at: http://127.0.0.1:5000/#/experiments/233217596645795621/runs/a87156801de24d5a89b1231327894650
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/233217596645795621


In [14]:
print(f"Model logged with run_id: {model_info.run_id}")

Model logged with run_id: ade8f706fb904cf9bf50fef5abed6a03


In [15]:
import mlflow.sklearn
from sklearn.datasets import load_iris

# Assuming you've already logged a model with the artifact path "iris_model"
# You need the run_id for the specific run you want to load the model from.
# Replace "<your_run_id>" with the actual run ID from your MLflow UI.
model_uri = "runs:/ade8f706fb904cf9bf50fef5abed6a03/iris_model" 

# Load the model
loaded_model = mlflow.sklearn.load_model(model_uri)

# Now you can use the loaded model for predictions
iris = load_iris()
X_test = iris.data # Use some test data
predictions = loaded_model.predict(X_test)
print(predictions)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 2 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 1 2 2 2 2
 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2
 2 2]


In [7]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

In [8]:
target_names = lr.classes_
target_names

array([0, 1, 2])

In [16]:
### MLFLOW tracking
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

##create a new MLFLOW experiment
mlflow.set_experiment("ML217")
## Start an MLFLOW run
with mlflow.start_run():
    ## log the hyperparameters
    mlflow.log_params(params)

    ## Log the accuracy metrics
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # --- NEW: CREATE AND LOG ARTIFACTS ---
    
    # 1. Create and save a Confusion Matrix plot
    predictions = lr.predict(X_test)
    cm = confusion_matrix(y_test, predictions)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()
    plt.title("Confusion Matrix")
    cm_plot_path = "confusion_matrix.png"
    plt.savefig(cm_plot_path)
    plt.close() # Important: close the plot to free memory
    # target_names = lr.classes_
    target_names = ['setosa', 'versicolor', 'virginica']
    # Log the plot as an artifact
    mlflow.log_artifact(cm_plot_path)

    # 2. Create and save a Classification Report text file
    report = classification_report(y_test, predictions, target_names=target_names, output_dict=False)
    report_path = "classification_report.txt"
    with open(report_path, 'w') as f:
        f.write(report)

    # Log the text report as an artifact
    mlflow.log_artifact(report_path)

    # 3. (Optional) Log the training data as an artifact for reference
    # X_train.to_csv("training_data_sample.csv", index=False)
    # mlflow.log_artifact("training_data_sample.csv")

    ## Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    ## log the model (This also logs the model as an artifact under the 'iris_model' path)
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        name="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="mybestmodel",
    )

    print(f"Model logged at: {model_info.model_uri}")

Registered model 'mybestmodel' already exists. Creating a new version of this model...
2025/08/26 16:59:19 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: mybestmodel, version 2
Created version '2' of model 'mybestmodel'.


Model logged at: models:/m-137670f5ee6a47c0bc17ecc76828bd67
🏃 View run victorious-cub-613 at: http://127.0.0.1:5000/#/experiments/974337509686012302/runs/537ef124277d4367bebe7f84ff06d397
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/974337509686012302


In [10]:
print(f"Model logged at: {model_info.model_uri}")

Model logged at: models:/m-8e0cc83d4d734c6aa1c071e919053916


In [17]:
## Inferencing from model from model registry

import mlflow.sklearn
model_name="mybestmodel"
model_version="latest"

model_uri=f"models:/{model_name}/{model_version}"

model=mlflow.sklearn.load_model(model_uri)
model

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,8888
,solver,'lbfgs'
,max_iter,1000
,multi_class,'auto'


In [18]:
y_pred_new=model.predict(X_test)
y_pred_new

array([2, 1, 0, 1, 1, 1, 1, 1, 2, 1, 0, 2, 0, 2, 0, 0, 1, 2, 2, 0, 0, 2,
       2, 0, 2, 1, 0, 1, 2, 1])

In [9]:
# Define the model hyperparameters
params = {"solver": "newton-cg", "max_iter": 1000, "multi_class": "auto", "random_state": 1000}

##train the model

lr=LogisticRegression(**params)
lr.fit(X_train,y_train)


e:\UDemy Final\MlFlowStarter\venv\lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='auto', random_state=1000,
                   solver='newton-cg')

In [10]:
y_pred=lr.predict(X_test)
y_pred

array([0, 2, 2, 1, 2, 1, 2, 2, 1, 2, 1, 2, 1, 1, 1, 0, 1, 2, 0, 2, 0, 2,
       1, 0, 0, 2, 1, 1, 0, 2])

In [11]:
accuracy=accuracy_score(y_test,y_pred)
print(accuracy)

0.9


In [12]:
## Sstart an MLFLOW run
with mlflow.start_run():
    ## log the hyperparameters
    mlflow.log_params(params)

    ## Log the accuracy metrics
    mlflow.log_metric("accuracy",accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    ## Infer the model signature

    signature=infer_signature(X_train,lr.predict(X_train))

    ## log the model
    model_info=mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart",

    )

Registered model 'tracking-quickstart' already exists. Creating a new version of this model...
2024/09/26 19:23:14 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 2
Created version '2' of model 'tracking-quickstart'.
2024/09/26 19:23:15 INFO mlflow.tracking._tracking_service.client: 🏃 View run calm-dolphin-198 at: http://127.0.0.1:5000/#/experiments/673599227836748446/runs/46ce7d9f08d941bca3d4108aa511ba15.
2024/09/26 19:23:15 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/673599227836748446.


In [9]:
model_info.model_uri

'models:/m-ba2ffd364a784f228ed3ce71de88b560'

## Inferencing And Validating Model

In [10]:
from mlflow.models import validate_serving_input

model_uri = model_info.model_uri

# The model is logged with an input example. MLflow converts
# it into the serving payload format for the deployed model endpoint,
# and saves it to 'serving_input_payload.json'
serving_payload = """{
  "inputs": [
    [
      5.7,
      3.8,
      1.7,
      0.3
    ],
    [
      4.8,
      3.4,
      1.6,
      0.2
    ],
    [
      5.6,
      2.9,
      3.6,
      1.3
    ],
    [
      5.4,
      3.7,
      1.5,
      0.2
    ],
    [
      6.7,
      3.3,
      5.7,
      2.5
    ],
    [
      6.5,
      3.0,
      5.8,
      2.2
    ],
    [
      5.1,
      2.5,
      3.0,
      1.1
    ],
    [
      7.7,
      3.0,
      6.1,
      2.3
    ],
    [
      5.2,
      4.1,
      1.5,
      0.1
    ],
    [
      5.0,
      3.3,
      1.4,
      0.2
    ],
    [
      4.6,
      3.6,
      1.0,
      0.2
    ],
    [
      4.5,
      2.3,
      1.3,
      0.3
    ],
    [
      5.6,
      2.8,
      4.9,
      2.0
    ],
    [
      6.2,
      3.4,
      5.4,
      2.3
    ],
    [
      6.3,
      3.4,
      5.6,
      2.4
    ],
    [
      4.8,
      3.4,
      1.9,
      0.2
    ],
    [
      5.8,
      2.7,
      5.1,
      1.9
    ],
    [
      7.2,
      3.0,
      5.8,
      1.6
    ],
    [
      5.4,
      3.9,
      1.7,
      0.4
    ],
    [
      6.9,
      3.2,
      5.7,
      2.3
    ],
    [
      6.5,
      3.0,
      5.5,
      1.8
    ],
    [
      7.7,
      2.6,
      6.9,
      2.3
    ],
    [
      6.4,
      3.1,
      5.5,
      1.8
    ],
    [
      5.2,
      3.5,
      1.5,
      0.2
    ],
    [
      6.3,
      2.9,
      5.6,
      1.8
    ],
    [
      4.9,
      3.1,
      1.5,
      0.2
    ],
    [
      7.9,
      3.8,
      6.4,
      2.0
    ],
    [
      5.5,
      4.2,
      1.4,
      0.2
    ],
    [
      6.1,
      3.0,
      4.9,
      1.8
    ],
    [
      5.0,
      3.2,
      1.2,
      0.2
    ],
    [
      4.9,
      3.0,
      1.4,
      0.2
    ],
    [
      6.0,
      2.2,
      4.0,
      1.0
    ],
    [
      5.0,
      3.6,
      1.4,
      0.2
    ],
    [
      4.6,
      3.4,
      1.4,
      0.3
    ],
    [
      6.9,
      3.1,
      5.1,
      2.3
    ],
    [
      5.0,
      3.5,
      1.3,
      0.3
    ],
    [
      5.8,
      2.8,
      5.1,
      2.4
    ],
    [
      4.9,
      3.1,
      1.5,
      0.1
    ],
    [
      5.6,
      2.7,
      4.2,
      1.3
    ],
    [
      7.0,
      3.2,
      4.7,
      1.4
    ],
    [
      6.3,
      2.7,
      4.9,
      1.8
    ],
    [
      5.0,
      3.4,
      1.5,
      0.2
    ],
    [
      5.8,
      2.7,
      5.1,
      1.9
    ],
    [
      6.1,
      2.9,
      4.7,
      1.4
    ],
    [
      7.2,
      3.6,
      6.1,
      2.5
    ],
    [
      6.7,
      3.1,
      5.6,
      2.4
    ],
    [
      6.3,
      3.3,
      6.0,
      2.5
    ],
    [
      6.3,
      2.5,
      4.9,
      1.5
    ],
    [
      4.8,
      3.1,
      1.6,
      0.2
    ],
    [
      6.4,
      2.8,
      5.6,
      2.1
    ],
    [
      6.2,
      2.9,
      4.3,
      1.3
    ],
    [
      5.7,
      2.5,
      5.0,
      2.0
    ],
    [
      6.0,
      3.4,
      4.5,
      1.6
    ],
    [
      5.2,
      3.4,
      1.4,
      0.2
    ],
    [
      5.7,
      4.4,
      1.5,
      0.4
    ],
    [
      5.0,
      2.3,
      3.3,
      1.0
    ],
    [
      6.2,
      2.2,
      4.5,
      1.5
    ],
    [
      6.3,
      2.3,
      4.4,
      1.3
    ],
    [
      6.3,
      2.8,
      5.1,
      1.5
    ],
    [
      5.1,
      3.3,
      1.7,
      0.5
    ],
    [
      6.0,
      2.9,
      4.5,
      1.5
    ],
    [
      6.5,
      3.0,
      5.2,
      2.0
    ],
    [
      5.3,
      3.7,
      1.5,
      0.2
    ],
    [
      6.8,
      2.8,
      4.8,
      1.4
    ],
    [
      5.9,
      3.0,
      5.1,
      1.8
    ],
    [
      4.7,
      3.2,
      1.6,
      0.2
    ],
    [
      5.5,
      3.5,
      1.3,
      0.2
    ],
    [
      6.7,
      3.0,
      5.2,
      2.3
    ],
    [
      5.7,
      2.6,
      3.5,
      1.0
    ],
    [
      5.8,
      2.7,
      4.1,
      1.0
    ],
    [
      5.5,
      2.5,
      4.0,
      1.3
    ],
    [
      5.1,
      3.8,
      1.9,
      0.4
    ],
    [
      5.7,
      2.8,
      4.5,
      1.3
    ],
    [
      6.7,
      3.1,
      4.4,
      1.4
    ],
    [
      6.4,
      3.2,
      5.3,
      2.3
    ],
    [
      7.7,
      2.8,
      6.7,
      2.0
    ],
    [
      5.5,
      2.6,
      4.4,
      1.2
    ],
    [
      5.7,
      2.9,
      4.2,
      1.3
    ],
    [
      7.4,
      2.8,
      6.1,
      1.9
    ],
    [
      7.6,
      3.0,
      6.6,
      2.1
    ],
    [
      6.3,
      3.3,
      4.7,
      1.6
    ],
    [
      5.4,
      3.0,
      4.5,
      1.5
    ],
    [
      6.2,
      2.8,
      4.8,
      1.8
    ],
    [
      4.4,
      3.2,
      1.3,
      0.2
    ],
    [
      6.6,
      2.9,
      4.6,
      1.3
    ],
    [
      7.2,
      3.2,
      6.0,
      1.8
    ],
    [
      4.7,
      3.2,
      1.3,
      0.2
    ],
    [
      5.7,
      2.8,
      4.1,
      1.3
    ],
    [
      5.5,
      2.3,
      4.0,
      1.3
    ],
    [
      5.1,
      3.8,
      1.5,
      0.3
    ],
    [
      6.1,
      2.6,
      5.6,
      1.4
    ],
    [
      4.6,
      3.2,
      1.4,
      0.2
    ],
    [
      6.7,
      3.3,
      5.7,
      2.1
    ],
    [
      6.7,
      3.1,
      4.7,
      1.5
    ],
    [
      5.6,
      2.5,
      3.9,
      1.1
    ],
    [
      5.1,
      3.5,
      1.4,
      0.2
    ],
    [
      5.0,
      3.4,
      1.6,
      0.4
    ],
    [
      5.1,
      3.5,
      1.4,
      0.3
    ],
    [
      4.8,
      3.0,
      1.4,
      0.1
    ],
    [
      6.1,
      2.8,
      4.7,
      1.2
    ],
    [
      5.9,
      3.0,
      4.2,
      1.5
    ],
    [
      5.0,
      3.5,
      1.6,
      0.6
    ],
    [
      6.1,
      3.0,
      4.6,
      1.4
    ],
    [
      6.8,
      3.0,
      5.5,
      2.1
    ],
    [
      5.1,
      3.8,
      1.6,
      0.2
    ],
    [
      5.7,
      3.0,
      4.2,
      1.2
    ],
    [
      4.4,
      2.9,
      1.4,
      0.2
    ],
    [
      4.4,
      3.0,
      1.3,
      0.2
    ],
    [
      5.4,
      3.4,
      1.5,
      0.4
    ],
    [
      7.7,
      3.8,
      6.7,
      2.2
    ],
    [
      5.0,
      2.0,
      3.5,
      1.0
    ],
    [
      6.4,
      2.9,
      4.3,
      1.3
    ],
    [
      6.0,
      2.2,
      5.0,
      1.5
    ],
    [
      4.6,
      3.1,
      1.5,
      0.2
    ],
    [
      5.8,
      2.7,
      3.9,
      1.2
    ],
    [
      4.9,
      2.5,
      4.5,
      1.7
    ],
    [
      4.8,
      3.0,
      1.4,
      0.3
    ],
    [
      5.4,
      3.9,
      1.3,
      0.4
    ],
    [
      6.4,
      3.2,
      4.5,
      1.5
    ],
    [
      6.3,
      2.5,
      5.0,
      1.9
    ]
  ]
}"""

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

array([0, 0, 1, 0, 2, 2, 1, 2, 0, 0, 0, 0, 2, 2, 2, 0, 2, 2, 0, 2, 2, 2,
       2, 0, 2, 0, 2, 0, 2, 0, 0, 1, 0, 0, 2, 0, 2, 0, 1, 1, 2, 0, 2, 1,
       2, 2, 2, 1, 0, 2, 1, 2, 1, 0, 0, 1, 1, 1, 2, 0, 1, 2, 0, 1, 2, 0,
       0, 2, 1, 1, 1, 0, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 2, 0, 1, 2, 0, 1,
       1, 0, 2, 0, 2, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 2, 0, 1, 0, 0, 0, 2,
       1, 1, 2, 0, 1, 1, 0, 0, 1, 2])

## Load the model back for prediction as a generic python function model

In [11]:
loaded_model=mlflow.pyfunc.load_model(model_info.model_uri)
predictions=loaded_model.predict(X_test)

iris_features_name=datasets.load_iris().feature_names

result=pd.DataFrame(X_test,columns=iris_features_name)
result["actual_class"]=y_test
result["predcited_class"]=predictions

In [12]:
result[:5]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),actual_class,predcited_class
0,6.1,2.8,4.7,1.2,1,1
1,6.4,3.2,4.5,1.5,1,1
2,5.5,4.2,1.4,0.2,0,0
3,4.3,3.0,1.1,0.1,0,0
4,5.0,3.2,1.2,0.2,0,0


### Model Registry

The MLflow Model Registry component is a centralized model store, set of APIs, and UI, to collaboratively manage the full lifecycle of an MLflow Model. It provides model lineage (which MLflow experiment and run produced the model), model versioning, model aliasing, model tagging, and annotations.

In [ ]:
## log the model
model_info=mlflow.sklearn.log_model(
    sk_model=lr,
    artifact_path="iris_model",
    signature=signature,
    input_example=X_train,
    registered_model_name="tracking-quickstart",

)

In [25]:
##create a new MLFLOW experiment
mlflow.set_experiment("MLFLOW Quickstart")

## Sstart an MLFLOW run
with mlflow.start_run():
    ## log the hyperparameters
    mlflow.log_params(params)

    ## Log the accuracy metrics
    mlflow.log_metric("accuracy",1.0)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info2", "Basic LR model for iris data")

    ## Infer the model signature

    signature=infer_signature(X_train,lr.predict(X_train))

    ## log the model
    ## log the model
    model_info=mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,

    )

2024/09/27 17:00:51 INFO mlflow.tracking._tracking_service.client: 🏃 View run judicious-dove-672 at: http://127.0.0.1:5000/#/experiments/673599227836748446/runs/a6a627446cfd4ad786f1597638fcc365.
2024/09/27 17:00:51 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/673599227836748446.


In [26]:
## Inferencing from model from model registry

import mlflow.sklearn
model_name="tracking-quickstart"
model_version="latest"

model_uri=f"models:/{model_name}/{model_version}"

model=mlflow.sklearn.load_model(model_uri)
model

e:\UDemy Final\MlFlowStarter\venv\lib\site-packages\mlflow\store\artifact\utils\models.py:31: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


LogisticRegression(max_iter=1000, multi_class='auto', random_state=1000,
                   solver='newton-cg')

In [27]:
model_uri

'models:/tracking-quickstart/latest'

In [28]:
y_pred_new=model.predict(X_test)
y_pred_new

array([0, 2, 2, 1, 2, 1, 2, 2, 1, 2, 1, 2, 1, 1, 1, 0, 1, 2, 0, 2, 0, 2,
       1, 0, 0, 2, 1, 1, 0, 2])